In [2]:
import numpy as np
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense
from sklearn.model_selection import train_test_split
from tensorflow.keras.callbacks import TensorBoard
import datetime
import pickle
print('Versão do TensorFlow: ', tf.__version__)
import platform

Versão do TensorFlow:  2.16.1


In [3]:
tf.config.list_physical_devices()


[PhysicalDevice(name='/physical_device:CPU:0', device_type='CPU'),
 PhysicalDevice(name='/physical_device:GPU:0', device_type='GPU')]

In [4]:

os = platform.system()
path = './combined_data.pkl'
    
if os == 'Darwin':  # 'Darwin' é o nome do kernel do macOS
    path = '../../datas/combined_data.pkl'
    
 
os = platform.system()
path = './combined_data.pkl'
log_dir = "../../logs/angle/fit/" + datetime.datetime.now().strftime("%Y%m%d-%H%M%S")
save_model = "./models/angle/combined_data.h5"
save_model_keras = "./models/angle/combined_data.keras"

if os == 'Darwin':  # 'Darwin' é o nome do kernel do macOS
    path = '../../datas/angle_datas.pkl'
    save_model = "../../models/angle/combined_data.h5"
    save_model_keras = "../../models/angle/combined_data.keras"

In [6]:


# Carregar os dados do arquivo .pkl
with open(path, 'rb') as f:
    X_train, y_train, X_test, y_test = pickle.load(f)

In [7]:
%load_ext tensorboard
log_dir = "../../logs/angle/fit/" + datetime.datetime.now().strftime("%Y%m%d-%H%M%S")
tensorboard_callback = TensorBoard(log_dir=log_dir, histogram_freq=1)

# Construir o modelo
model = tf.keras.Sequential([
    tf.keras.layers.Dense(128, activation='relu', input_shape=(5,)),  # Ajustar a entrada para 3 features
    tf.keras.layers.Dense(64, activation='relu'),  # Ajustar a entrada para 3 features
    tf.keras.layers.Dense(6, activation='softmax')  #  
])

# Compilar o modelo
model.compile(optimizer='adam', loss='sparse_categorical_crossentropy', metrics=['accuracy'])

# Treinar o modelo
model.fit(np.array(X_train), np.array(y_train), epochs=5000, batch_size=10000, validation_split=0.2, callbacks=[tensorboard_callback])


The tensorboard extension is already loaded. To reload it, use:
  %reload_ext tensorboard
Epoch 1/5000
116/116 ━━━━━━━━━━━━━━━━━━━━ 2s 11ms/step - accuracy: 0.7669 - loss: 26.2678 - val_accuracy: 0.9673 - val_loss: 0.4933
Epoch 2/5000
116/116 ━━━━━━━━━━━━━━━━━━━━ 1s 9ms/step - accuracy: 0.9692 - loss: 0.3056 - val_accuracy: 0.9763 - val_loss: 0.1229
Epoch 3/5000
116/116 ━━━━━━━━━━━━━━━━━━━━ 1s 9ms/step - accuracy: 0.9768 - loss: 0.1085 - val_accuracy: 0.9805 - val_loss: 0.0804
Epoch 4/5000
116/116 ━━━━━━━━━━━━━━━━━━━━ 1s 9ms/step - accuracy: 0.9805 - loss: 0.0721 - val_accuracy: 0.9820 - val_loss: 0.0566
Epoch 5/5000
116/116 ━━━━━━━━━━━━━━━━━━━━ 1s 9ms/step - accuracy: 0.9826 - loss: 0.0538 - val_accuracy: 0.9840 - val_loss: 0.0462
Epoch 6/5000
116/116 ━━━━━━━━━━━━━━━━━━━━ 1s 9ms/step - accuracy: 0.9844 - loss: 0.0437 - val_accuracy: 0.9859 - val_loss: 0.0391
Epoch 7/5000
116/116 ━━━━━━━━━━━━━━━━━━━━ 1s 9ms/step - accuracy: 0.9859 - loss: 0.0377 - val_accuracy: 0.9869 - val_loss: 0.034

In [ ]:
# Avaliar o modelo
loss, accuracy = model.evaluate(np.array(X_test), np.array(y_test))
print(f'Model Accuracy: {accuracy * 100:.2f}%')

In [ ]:
# Ações
# 0 Nothing
# 1 Girar para a Esquerda
# 2 Girar para a Direita

# GO TO RIGHT
angle = 180
 
class_names = ['Nothing', 'RoteteLeft', 'RotateRight', ]
predictions = model.predict(np.array([ angle]))
 
score = tf.nn.softmax(predictions[0])
print(  "{}-Must be GO TO RIGHT: {} - with a {:.2f} percent confidence."
    .format(np.argmax(score),class_names[np.argmax(score)], 100 * np.max(score))
)
 



In [ ]:
%tensorboard

In [ ]:
model.save(save_model)
model.save(filepath=save_model_keras)

### From here to you only MACOS to test 

In [ ]:

# !pip install tensorflowjs
from tensorflow.keras.models import load_model
import tensorflow as tf

# Carregar o modelo do arquivo .h5
model = load_model('../../models/angle/model_saved.h5')

# Compile o modelo (você pode precisar ajustar a função de perda, otimizador e métricas conforme necessário)
model.compile(optimizer='adam', loss='sparse_categorical_crossentropy', metrics=['accuracy'])

# Salvar o modelo no formato SavedModel
tf.saved_model.save(model, '../../models/angle/saved_model/')

!tensorflowjs_converter --input_format=tf_saved_model --output_format=tfjs_graph_model '../../models/angle/saved_model/' ../../models/tf/angle/web_model